### **Импортируемые бибилиотеки и модули. Не трогать.**

In [ ]:
# Уставнавливаем дополнительные модули и пакету через Python Index Package (PIP)
!pip install advertools
!pip install emot

# Подключаемые модули и библиотеки для обработки 
import pandas as pd
pd.options.display.max_columns = None
import numpy as np
import requests
import nltk
import math
import time
import os

# Для моделирования и настройки нейронной сети 
from keras.utils import np_utils
from keras.initializers import RandomNormal
from keras.initializers import he_normal
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Dropout

# Для работы с парсингом и токенизированию слов
from io import BytesIO
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Для обработки текстовых данных
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
from sklearn.metrics import pairwise_distances_argmin_min
from scipy.sparse import hstack
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.callbacks import ModelCheckpoint

# Использование фреймворка TensorFlow для обработки текстовых данных
import tensorflow as tf
import tensorflow_hub as hub

# Визуализация данных и предупреждения
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use(style = "tableau-colorblind10")
plt.rcParams['figure.figsize'] = (15, 10)
%matplotlib inline
import plotly
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot , plot
init_notebook_mode(connected = True)

# Библиотека advertools для работы с рекламой и ключевыми словами
# https://pypi.org/project/advertools/
# https://advertools.readthedocs.io/en/master/advertools.html#module-advertools.serp
# https://pypi.org/project/textgenrnn/
import advertools as adv

# Прочее
import warnings
warnings.filterwarnings("ignore")
from google.colab import files

     |████████████████████████████████| 245kB 2.8MB/s 
     |████████████████████████████████| 245kB 8.7MB/s 
     |████████████████████████████████| 3.1MB 11.8MB/s 
     |████████████████████████████████| 235kB 22.0MB/s 
     |████████████████████████████████| 2.7MB 28.4MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 3.2MB 43.0MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11515 sha256=a1d002a332996c540c147fd8d2430770aff45bad6c6fe719ae4c54355a7c3c51
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
  Created wheel for protego: filename=Protego-0.1.16-cp36-none-any.whl size=7765 sha256=8f3fe773c78d4cbd006d0a9c6d645ad114ea8c2c4595c838c9cf339687d87a21
  Stored in directory: /root/.cache/pip/wheels/51/01/d1/4a2286a976dccd025ba679acacfe37320540df0f2283ecab12
Successfully built PyDispatc

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


> Если вдруг обновится какой-либо из модулей и/или данный **код будет запускаться у другого пользователя**, то потребуется обновление/установка модулей/фреймворков, так как **Google Colab не обновляет модули в автоматическом режиме**. Используем следующее, в **строчку кода вводим режим обновления/установки:**
---
```
# Python >= 3.6.6
!pip install <название_модуля>
```



### **Загружаем данные с Google Drive для работы с ними.**

> Для того, чтобы работать в данном облаке необходимо чтобы в личном аккаунте Google Drive была папка с данным проектом. Внутри папки располагаем дополнительную папку с набором данным для последующего использования. С запуском кода ниже, потребуется ввести креды для своего аккаунта, где находятся данные файлы. 
---
Папка на диске должна выглядеть следующим образом:
https://prnt.sc/td9v11


In [ ]:
# На данном этапе потребуется авторизация аккаунта и Google SDK для работы
# Запускаем код ниже и авторизуемся и вводим в строку Credentials код присланный Google
!pip install -U -q PyDrive

In [ ]:
# Код для загрузки данных с диска Google для работы с ними в текущем окружении.
# Прежде требуется установить модуль !pip install -U -q PyDrive (уже установлен).
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Авторизация и создание клиента
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive()

In [ ]:
# Загрузка файлов через Google SDK
# Необходимо иметь данные на локальном диске и при нажатии кнопки загрузить выбрать их всех для загрузки
# Начнется процесс загрузки данных в облако для работы, далее их еще необходимо будет сохранить в разные перменные среды.
from google.colab import files
uploaded = files.upload()

Saving CompetitorsUS_keywords.csv to CompetitorsUS_keywords.csv
Saving Ahrefs_keywords.csv to Ahrefs_keywords.csv
Saving Adwords_keywords.csv to Adwords_keywords.csv
Saving GSC_keywords.csv to GSC_keywords.csv


In [ ]:
# Загружаем все данные в разные переменные среды для работы с ними
adwords_data = pd.read_csv(BytesIO(uploaded['Adwords_keywords.csv']))
ahrefs_data = pd.read_csv(BytesIO(uploaded['Ahrefs_keywords.csv']))
gsc_data = pd.read_csv(BytesIO(uploaded['GSC_keywords.csv']))
competitorUS_data = pd.read_csv(BytesIO(uploaded['CompetitorsUS_keywords.csv']))

### **Работа с данными. Очистка.**

In [ ]:
# Информация про размерность данных
print(' adwords_data: {}\n ahrefs_data: {}\n gsc_data: {}\n competitorUSA_data: {}'.format(adwords_data.shape, ahrefs_data.shape, gsc_data.shape, competitorUS_data.shape), end = ' ')

 adwords_data: (21172, 3)
 ahrefs_data: (26543, 1)
 gsc_data: (384287, 3)
 competitorUSA_data: (144905, 1) 

In [ ]:
# Информация о фичах в данных
print('Columns/Features info:\n {}\n {}\n {}\n {}\n'.format(adwords_data.columns, ahrefs_data.columns, gsc_data.columns, competitorUS_data.columns))

Columns/Features info:
 Index(['Matched search term', 'Impressions', 'Clicks'], dtype='object')
 Index(['women 4men in cookeville'], dtype='object')
 Index(['Search query', 'Impressions', 'Clicks'], dtype='object')
 Index(['interracial dating'], dtype='object')



In [ ]:
# Сохраняем копии данных и избавляемся от ненужных признаков, оставляем только keyword sequence
# так как нам еще понадобятся признаки в основных таблицах - Impressions и Clicks для набора ключевых слов
adwords = adwords_data.copy(deep = False)
ahrefs = ahrefs_data.copy(deep = False)
gcs = gsc_data.copy(deep = False)
competitorUS = competitorUS_data.copy(deep = False)

In [ ]:
# Если всё False значит это отдельные объекты, с которыми можно работать. Первоначальные объекты / таблицы не изменяться.
print(" Copy validation checking:\n Adwords: {}\n Ahrefs: {}\n GSC: {}\n Competitors: {}\n".format(
       adwords is adwords_data, ahrefs is ahrefs_data, gcs is gsc_data, competitorUS is competitorUS_data
))

 Copy validation checking:
 Adwords: False
 Ahrefs: False
 GSC: False
 Competitors: False



In [ ]:
# Очистка данных Adwords
# Для работы с keywords необходимо удалить такие признаки в наборе как Impressions и Clicks
del(adwords['Impressions'])
del(adwords['Clicks'])
del(gcs['Clicks'])
del(gcs['Impressions'])

In [ ]:
# Устанавливаем одинаковые имена для столбцов данных чтобы можно было работать с ними в последующем
# Если в таблице уже есть признак который надо переименовать используем метода dataset.rename(columns = {'old_name' : 'new_name'}, inplace = True)
# Если в таблице отсутствует название признакак и надо его добавить используем метод: dataset.columns = ['name_of_the_column']
adwords.rename(columns = {'Matched search term' : 'keywords'}, inplace = True)
ahrefs.columns = ['keywords']
gcs.rename(columns = {'Search query' : 'keywords'}, inplace = True)
competitorUS.columns = ['keywords']

In [ ]:
# Информация о признаках в данных и изменились ли названия признаков
print('Columns/Features information:\n {}\n {}\n {}\n {}\n'.format(adwords.columns, ahrefs.columns, gcs.columns, competitorUS.columns))

Columns/Features information:
 Index(['keywords'], dtype='object')
 Index(['keywords'], dtype='object')
 Index(['keywords'], dtype='object')
 Index(['keywords'], dtype='object')



In [ ]:
# Соединяем все данные, где у нас содержаться keywords -> [adwords + ahrefs + gcs]
# в единый набор данных для того чтобы провести анализ ключевых слов и их комбинаций
# используемый метод pd.concat(__массив_всех_данных__)
# Здесь в дата эрей можно удалять и добавлять массивы данных
data_array = [adwords, ahrefs, gcs]
all_keywords = pd.concat(data_array)

In [ ]:
# Проверяем какое количество записей в наборе данных
# Всего должно получиться записей 432 002
print("Всего в наборе данных содержится {} записи.".format(len(all_keywords)))

Всего в наборе данных содержится 432002 записи.


In [ ]:
# Перевод всех данных в наборе в формат текста для того чтобы избежать ошибок связанных с типом данных
all_keywords['keywords'] = all_keywords['keywords'].astype(str)

In [ ]:
# Проверка есть ли еще дополнительные типы данных в наборе
# Всё должно быть в формате типа данных object
all_keywords.dtypes

keywords    object
dtype: object

In [ ]:
# Вычисляем максимальное и минимальное словосочетание (количество слов в keyword sequence)
print('Максимальное словосочетание в наборе keywords {} слов.'.format(all_keywords['keywords'].str.split().str.len().max()))
print('Минимальное словосочетание в наборе keywords {} слово.'.format(all_keywords['keywords'].str.split().str.len().min()))

Максимальное словосочетание в наборе keywords 45 слов.
Минимальное словосочетание в наборе keywords 1 слово.


In [ ]:
# Проверка на наличие пропущенных значений если они будут то их необходимо будет удалить
all_keywords['keywords'].isnull().value_counts()

False    432002
Name: keywords, dtype: int64

In [ ]:
# Для того чтобы работать дальше с текстом его необходимо немного преобразовать
# Первым делом переведем все слова находящиеся в колонке в нижний регистр
all_keywords['keywords'] = all_keywords['keywords'].str.lower()

In [ ]:
# Если в наборе keywords присутствует пунктуация от нее тоже необходимо будет избавиться для последующей работы с текстом
all_keywords['keywords'] = all_keywords['keywords'].str.replace('[^\w\s]', '')

In [ ]:
# В датасете существует большое количество смайликов (emoji) не представляющих никакой смысловой нагрузки для формирования keyword
# Необходимо будет удалить такие записи для того, чтобы провести текстовой анализ ключевых слов

import re     # библиотека регулярных выражений Python
import sys

# функция для удаления emoji из набора данных
def emoji(string):
  emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols and pictographs
                               u"\U0001F680-\U0001F6FF"  # transport and map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (for iOS)
                               u"\U00002500-\U00002BEF"  # chinese char (non ASCII characters)
                               u"\U00002702-\U000027B0"  
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags = re.UNICODE)
  return emoji_pattern.sub(r'', str(string))

In [ ]:
# Теперь удаляем все смайлики и прочую non-ASCII текст в наборе данных
all_keywords['keywords'] = all_keywords['keywords'].apply(emoji)

In [ ]:
# Так же в наборе данных после нашего преобразования присутствуют пустые значения, которые необходимо будет удалить
# Сначала проверка на наличие и действительно ли они существуют:
print('Количество пустых значений в наборе данных: {}'.format(len(all_keywords[all_keywords['keywords'] == ''])))

Количество пустых значений в наборе данных: 400


In [ ]:
# Удаляем такие значения и получаем готовый набор данных с которым можно работать
all_keywords = all_keywords[all_keywords['keywords'] != '']

In [ ]:
# Кроме того в наборе данных также необходимо удалить emoticons - выражения/подобия смайлов: :), )), xD и так далее
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

# Фунцкция для удаления эмотиконов
def remove_emoticons(text):
  emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
  return emoticon_pattern.sub(r'', text)

In [ ]:
# Вызываем вышестоящую функцию для удаления эмотиконов
all_keywords['keywords'] = all_keywords['keywords'].apply(remove_emoticons)

In [ ]:
# Кроме того в наборе данных присутствуют URL и адреса веб страниц, которые также необходимо будет удалить

# Функция для удаления адресов веб-страниц
def remove_urls(text):
  url_pattern = re.compile(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*')
  return url_pattern.sub(r'', text)

In [ ]:
# Вызываем вышеуказанную функцию и удаляем адреса веб-сайтов
all_keywords['keywords'] = all_keywords['keywords'].apply(remove_urls)

In [ ]:
# Избавляемся от цифр и символов в наборе
import string
all_keywords['keywords'] = all_keywords['keywords'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
all_keywords['keywords'] = all_keywords['keywords'].apply(lambda x: x.translate(str.maketrans('', '', '1234567890')))
all_keywords = all_keywords[all_keywords['keywords'] != ' ']
len(all_keywords)

431525

In [ ]:
# фильтр строк на пустые символы
all_keywords['keywords'].str.replace(' ', '')
all_keywords.keywords = all_keywords.keywords.str.strip()

# избавляемся от пустых строк
all_keywords = all_keywords[all_keywords['keywords'] != '']

# смотрим на количество строк в наборе данных
len(all_keywords)

431264

In [ ]:
# Так же удаляем Non-ASCII characters (другие языки кроме латиницы, если они есть в наборе данные)
def remove_nonASCII(text):
  nonASCII_pattern = re.compile(r"([^\x00-\x7F])+")
  return nonASCII_pattern.sub(r"", text)

In [ ]:
# Плюс для того чтобы удалить NON-ASCII characters можно использовать модуль от разработчиков Facebook - fasstext 
! pip install fasttext

     |████████████████████████████████| 71kB 2.1MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3016738 sha256=e5d24d1ccea855ca515ed32726cb1987eb0f58ff1143c523fa549cea07ce06fa
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
!wget -O /tmp/lid.176.bin https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin

--2020-07-21 12:35:40--  https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131266198 (125M) [application/octet-stream]
Saving to: ‘/tmp/lid.176.bin’

/tmp/lid.176.bin    100%[===================>] 125.18M  23.8MB/s    in 5.9s    

2020-07-21 12:35:46 (21.4 MB/s) - ‘/tmp/lid.176.bin’ saved [131266198/131266198]



In [ ]:
#  Используем для простановки меток 
import contextlib
import fasttext

# Нейросеть которая распознаёт язык lid.176.bin
pretrained_model = '/tmp/lid.176.bin'
model = fasttext.load_model(pretrained_model)

# Распознавание языков
langs = []
for sentence in all_keywords['keywords']:
  lang = model.predict(sentence)[0]
  langs.append(str(lang)[11:13])

all_keywords['language'] = langs

In [ ]:
# избавляемся от текста, который имеют языковую разметку != en
len(all_keywords[all_keywords['language'] != 'en'])

82033

In [ ]:
# набор данных, который содержит в себе только английские слова
all_keywords = all_keywords[all_keywords['language'] == 'en']

In [ ]:
# Сколько языковых групп находится в наборе данных
all_keywords['language'].value_counts()

en    349231
Name: language, dtype: int64

In [ ]:
# Удалям колонку с название языковой группы
del(all_keywords['language'])

### **Анализ ключевых слов в наборе данных**

In [ ]:
# Так как в наборе слишком много "лишних слов" таких как: a, the, and, in etc. от них необходимо избавиться так как смысловой нагрузки нет
# То есть опять приступаем к очистке данных в наборе keywords методом токенизирования
tokenized_sentences = all_keywords['keywords'].tolist()

In [ ]:
# Тоже самое только надо воспроизвести отдельны "единичные слова" в наборе keywords в наборе данных
tokenized_words = all_keywords['keywords'].str.split(' ').explode().str.strip().tolist()

In [ ]:
# Избавляемся от StopWords 
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
print(stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
{'being', 'until', 'she', 'yours', 'which', 'over', 'can', 'hadn', 'any', 'how', 'their', 'both', 'were', 'have', "haven't", 'i', 'for', "it's", 'during', 'few', 'ain', "isn't", 'up', 'is', "shouldn't", 'am', 'further', 'doing', 'couldn', 'same', 've', 'been', 'each', 'no', 'once', 'theirs', 'did', 'more', "should've", 'between', 'again', 'then', 't', 'weren', 'y', 'wouldn', 'will', 'are', 'just', 'where', 'about', 'down', 'themselves', 'her', 'itself', 'mightn', "you're", 'in', "wouldn't", 'now', 'herself', 'them', 'that', 'had', 'shan', 'we', "she's", 'yourself', 'below', 'won', 'an', "you'd", 'myself', 'the', 'under', 'hasn', 'haven', 'nor', 'there', "won't", 'if', 'you', 's', 'he', 'above', 'only', 'some', 'too', 'of', "aren't", 'most', 'o', 'wasn', 're', 'd', 'because', "shan't", 'your', 'against', 'should', "you've", 'other', 'needn', 'himself', 'what', 'after', 'ma', 'h

In [ ]:
# Необходимо добавить еще стоп слова, так как некоторые позиции отсутствуют в библиотеке
stop_words = {'a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren',
              "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can',
              'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during',
              'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having',
              'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's",
              'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn',
              "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over',
              'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn', "shouldn't", 'so', 'some', 'such', 't', 'than',
              'that', "that'll", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too',
              'under', 'until', 'up', 've', 'very', 'was', 'wasn', "wasn't", 'we', 'were', 'weren', "weren't", 'what', 'when', 'where', 'which', 'while', 'who',
              'whom', 'why', 'will', 'with', 'won', "won't", 'wouldn', "wouldn't", 'y', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself',
              'yourselves', 
              'in', 'for', 'а', 'how', 'the', 'of', 'xxx', 'XXX', 'com', ' com', 'com ', 'no', ' no', 'no ', 'me', ' me', 'me ',
              'is', ' is', 'is ', 'you', ' you', 'you ', 'are', ' are', 'are ', 'www', ' www', 'www ', 'the', ' the', 'the ', 'one ', 'one', ' one',
              'net', ' net', 'net ', 'by', ' by', 'by ', 'from ', ' from', ' he', 'he ', ' he ', 'her ', ' her', ' her ', 'it ', ' it', ' it ', ' as',
              'as ', ' as ', 'in ', ' in', ' in ', ' from ', 'to ', ' to', ' to ', 's', ' s', 's ', 'at', ' at', 'at ', 'on', 'on ', ' on'}

In [ ]:
import nltk
nltk.download('punkt')

# Убираем стоп слова с предложений по ключевым словам
filtered_sentences = []
for sentence in tokenized_sentences:
    filtered_sentences.append(' '.join(word for word in nltk.word_tokenize(sentence) if word.lower() not in stop_words))

filtered_sentences = [word for word in filtered_sentences if word]


# Убираем стоп слова с предложений по ключевым предложениям
filtered_words = []
for sentence in tokenized_words:
    filtered_words.append(''.join(word for word in nltk.word_tokenize(sentence) if word.lower() not in stop_words))

filtered_words = [word for word in filtered_words if word]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Теперь рассмотрим частотность всех слов в нашем наборе
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize, sent_tokenize

freq_dist = FreqDist(filtered_words)
print(freq_dist)

<FreqDist with 48276 samples and 1034833 outcomes>


In [ ]:
# # Stemming and Lemmatization
# from nltk.stem import PorterStemmer, LancasterStemmer, SnowballStemmer
# from nltk.tokenize import sent_tokenize, word_tokenize

# # Создаем экземпляр класса для PorterStemmer
# ps = PorterStemmer()
# lst = LancasterStemmer()
# englishStemmer = SnowballStemmer('english', ignore_stopwords = True)

# # Filtered and stemmed words
# stemmed_filtered_words = []

# for w in filtered_words:
#   stemmed_filtered_words.append(englishStemmer.stem(w))

# print("Filtered Words: ", filtered_words[:20])
# print("Stemmed Filtered Words: ", stemmed_filtered_words[:20])

In [ ]:
# Повторный расчет набора слов в keywords наборе данных
freq_dist = FreqDist(filtered_words)

# Расчет для частотности предложений (целых keywords) в наборе данных
freq_SentDist = FreqDist(filtered_sentences)

In [ ]:
# Фильтр количества словосочетаний в наборе ключевых слов в наборе даных
def structure_keyword(tokenizedArray, lenght, emptyArray):
  for keyword in tokenizedArray:
    if len(keyword.split(' ')) == lenght:
      emptyArray.append(keyword)

In [ ]:
# Создаем массивы для ключевых слов. Для 2, 3, 4 и 5 словосочетаний.
tokenized_twoWords = []
tokenized_threeWords = []
tokenized_fourWords = []
tokenized_fiveWords = []
tokenized_sixWords = []
tokenized_sevenWords = []
tokenized_eightWords = []
tokenized_nineWords = []
tokenized_tenWords = []

# Вызываем функцию для поиска и заполняем массив
structure_keyword(filtered_sentences, 2, tokenized_twoWords)
structure_keyword(filtered_sentences, 3, tokenized_threeWords)
structure_keyword(filtered_sentences, 4, tokenized_fourWords)
structure_keyword(filtered_sentences, 5, tokenized_fiveWords)
structure_keyword(filtered_sentences, 6, tokenized_sixWords)
structure_keyword(filtered_sentences, 7, tokenized_sevenWords)
structure_keyword(filtered_sentences, 8, tokenized_eightWords)
structure_keyword(filtered_sentences, 9, tokenized_nineWords)
structure_keyword(filtered_sentences, 10, tokenized_tenWords)

In [ ]:
print(' Объем каждого из массивов:\n 1 keywords: {}\n 2 keywords: {}\n 3 keywords: {}\n 4 keywords: {}\n 5 keywords: {}\n 6 keywords: {}\n 7 keywords: {}\n 8 keywords: {}\n 9 keywords: {}\n 10 keywords: {}\n'.format(len(filtered_words),len(tokenized_twoWords),
                                                                                                                 len(tokenized_threeWords),
                                                                                                                 len(tokenized_fourWords),
                                                                                                                 len(tokenized_fiveWords),
                                                                                                                 len(tokenized_sixWords),
                                                                                                                 len(tokenized_sevenWords),
                                                                                                                 len(tokenized_eightWords),
                                                                                                                 len(tokenized_nineWords),
                                                                                                                 len(tokenized_tenWords)))

 Объем каждого из массивов:
 1 keywords: 1034833
 2 keywords: 94449
 3 keywords: 145858
 4 keywords: 70360
 5 keywords: 16495
 6 keywords: 2922
 7 keywords: 702
 8 keywords: 180
 9 keywords: 79
 10 keywords: 54



In [ ]:
# Расчет частотности:
freqDist_oftwoWords = FreqDist(tokenized_twoWords)
freqDist_ofthreeWords = FreqDist(tokenized_threeWords)
freqDist_offourWords = FreqDist(tokenized_fourWords)
freqDist_offiveWords = FreqDist(tokenized_fiveWords)
freqDist_ofsixWords =  FreqDist(tokenized_sixWords)

# Смотрим что содержится в массивах
print(tokenized_twoWords[:10])
print('==================')
print(tokenized_threeWords[:10])
print('==================')
print(tokenized_fourWords[:10])
print('==================')
print(tokenized_fiveWords[:10])
print('==================')
print(tokenized_sixWords[:10])

['dating sites', 'plenty fish', 'online dating', 'meetville app', 'dating site', 'meetville review', 'girls near', 'chat rooms', 'dating websites', 'meetville reviews']
['online dating websites', 'online dating site', 'free dating sites', 'online dating sites', 'free chat rooms', 'online dating service', 'dating web sites', 'meetville free subscription', 'meetville dating site', 'free dating site']
['talk female strangers online', 'best dating app uae', 'best dating app dubai', 'percent free dating sites', 'get free chat meetville', 'meetville free subscription hack', 'meet single asians online', 'get meetville subscription free', 'dating sites free membership', 'free dating sites men']
['meetville chat date local singles', 'online free dating site usa', 'free online local dating sites', 'real online dating sites free', 'meetville premium apk free download', 'totally free online dating sites', 'best free online dating sites', 'christian dating sites free browsing', 'free online dating 

In [ ]:
# Экспортируем сгенеренные результаты в папку (прежде надо смонтировать connection с гугл-диском)
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [ ]:
export_data = [freq_dist, freqDist_oftwoWords, freqDist_ofthreeWords, freqDist_offourWords, freqDist_offiveWords, freqDist_ofsixWords]

# Экспортируем полученные результаты по частотности в файлы csv для передачи отделам
# Указываем путь экспорта файла и количество ключевых слов для выгрузки
def to_export(data_to_export):
  for num, document in enumerate(data_to_export):
    export_doc = pd.DataFrame(document.most_common(100), columns = ['keywords', 'frequency']).sort_values(by = ['frequency'], ascending = False)
    export_doc.to_csv('drive/My Drive/Colab Notebooks/KeywordsAnalytics_KeywordsModelling/Exported_keywordsFrequency_UniqueSet/'+ str(num) + '_documentFreqency.csv', index = False)

In [ ]:
# Экспортируем данные в папку
# Данные представляют с собой наиболее часто используемые однословники, двух... и ...шестисловники в наборе данных
# То есть, те keywords которые были использованны в процессе работы чаще всего.
to_export(export_data)

### **Формирование массива признаков ключевых слов, по частотности.Классификация результатов. Уникальное вхождение и не уникальное вхождение частотности в keywords.**

In [ ]:
# В случае классификации текста необходимо сконвертировать данный текст на некоторое количество векторов.
# В качестве признаков будем использовать наиболее частов встречающиеся слова, которые мы нашли в массиве tokenized_words
# Будем использовать Топ-50 слов с данного массива в результате получится матрица вида Nx50
# Какие массивы будут использоваться: freq_dist, tokenized_words and all keywords
bag_words = sorted(freq_dist.items(), key = lambda x: x[1], reverse = True)
print(bag_words)

[('dating', 56280), ('sex', 31085), ('girls', 23847), ('girl', 22224), ('women', 18534), ('call', 14973), ('men', 11591), ('sites', 11420), ('online', 11009), ('indian', 10772), ('gay', 10570), ('singles', 10337), ('number', 10179), ('kiss', 8918), ('free', 8678), ('aunty', 8498), ('site', 8416), ('single', 8246), ('date', 8211), ('love', 7697), ('man', 7514), ('woman', 7511), ('tamil', 6682), ('meet', 6618), ('chat', 6534), ('sexy', 5825), ('hot', 5447), ('old', 5092), ('meetville', 5073), ('mature', 4546), ('romantic', 4447), ('seeking', 4444), ('best', 4424), ('looking', 4368), ('first', 4320), ('lesbian', 4082), ('kissing', 3974), ('app', 3624), ('couple', 3617), ('photo', 3536), ('escort', 3477), ('near', 3446), ('escorts', 3418), ('relationship', 3405), ('black', 3391), ('india', 3313), ('bbw', 3228), ('ladies', 3192), ('images', 3150), ('phone', 3097), ('asian', 3096), ('photos', 3054), ('female', 2846), ('video', 2833), ('kerala', 2826), ('local', 2821), ('service', 2791), ('ba

In [ ]:
# Так как тип данных "словарное множество" то извлечем Топ-100 слов которые наиболее часто встречаются в запросах ключевых слов
# частотность опустим так как не совсем важна для последующих операций
bag_words = [keyword[0] for keyword in bag_words[:100]]

In [ ]:
adwords_data.rename(columns = {'Matched search term' : 'keywords', 'Impressions' : 'impressions', 'Clicks' : 'clicks'}, inplace = True)
gsc_data.rename(columns = {'Search query' : 'keywords', 'Impressions' : 'impressions', 'Clicks' : 'clicks'}, inplace = True)

In [ ]:
# Создаем проверяющий массив на сбор статистики по кликам и по impressions для нашего массива слов
checking_array = [adwords_data, gsc_data]
checking_data = pd.concat(checking_array, ignore_index = True, axis = 0)

In [ ]:
# Предобработка данных для checking data

# удаляем пустые значения
checking_data.dropna(inplace=True)

# Перевод всех данных в наборе в формат текста для того чтобы избежать ошибок связанных с типом данных
checking_data['keywords'] = checking_data['keywords'].astype(str)

# Переведем все слова находящиеся в колонке в нижний регистр
checking_data['keywords'] = checking_data['keywords'].str.lower()

# Если в наборе keywords присутствует пунктуация от нее тоже необходимо будет избавиться для последующей работы с текстом
checking_data['keywords'] = checking_data['keywords'].str.replace('[^\w\s]', '')

# Теперь удаляем все смайлики и прочую non-ASCII текст в наборе данных
checking_data['keywords'] = checking_data['keywords'].apply(emoji)

# Вызываем вышестоящую функцию для удаления эмотиконов
checking_data['keywords'] = checking_data['keywords'].apply(remove_emoticons)

# Вызываем вышеуказанную функцию и удаляем адреса веб-сайтов
checking_data['keywords'] = checking_data['keywords'].apply(remove_urls)

# Избавляемся от цифр и символов в наборе
checking_data['keywords'] = checking_data['keywords'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
checking_data['keywords'] = checking_data['keywords'].apply(lambda x: x.translate(str.maketrans('', '', '1234567890')))

# фильтр строк на пустые символы
checking_data['keywords'].str.replace(' ', '')
checking_data.keywords = checking_data.keywords.str.strip()

#  Используем для простановки меток 
# Нейросеть которая распознаёт язык lid.176.bin
pretrained_model = '/tmp/lid.176.bin'
model = fasttext.load_model(pretrained_model)

# Распознавание языков
langs = []
for sentence in checking_data['keywords']:
  lang = model.predict(sentence)[0]
  langs.append(str(lang)[11:13])

checking_data['language'] = langs

In [ ]:
checking_data.language.value_counts(normalize = True)[:10]

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


en    0.802004
it    0.027204
fr    0.024143
es    0.023544
de    0.023512
pt    0.007520
nl    0.007461
fi    0.006704
pl    0.005488
id    0.005174
Name: language, dtype: float64

In [ ]:
# Избавляемся от языка, который не EN
checking_data = checking_data[checking_data['language'] == 'en']
checking_data.language.value_counts()

en    325179
Name: language, dtype: int64

In [ ]:
# избавляемся от пустых строк
checking_data = checking_data[checking_data['keywords'] != '']

In [ ]:
# удаляем столбец language так как он больше не актуален
del(checking_data['language'])

In [ ]:
# Переводим обратно в строку
checking_data['keywords'] = checking_data['keywords'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop_words]))

In [ ]:
# Создаем аггрегированные данные по массиву частотности однословок по непрямому вхождению (по словам в отдельности)
a = []
for b in bag_words:
  a.append((b, checking_data[checking_data['keywords'].str.contains(b)][['impressions', 'clicks']].sum().values[0], 
               checking_data[checking_data['keywords'].str.contains(b)][['impressions', 'clicks']].sum().values[1]))

# набор агрегированных данные
groupedOne_df = pd.DataFrame.from_records(a, columns = ['keywords', 'impressions', 'clicks'])
groupedOne_df.head()

,keywords,impressions,clicks
0,dating,2980544,335716
1,sex,1448782,234933
2,girls,4523111,654593
3,girl,7400633,1084941
4,women,2380185,196865


In [ ]:
# группируем однословные частоты в набор и готово
groupedOne_df = groupedOne_df.sort_values(by = 'clicks', ascending = False)[:100]

In [ ]:
# То, что мы проводили выше с однословными предложениями теперь необходимо провести с двухсловными наборами 
# Так же получить количество кликов и количество impressions для набора данных
twoBag_words = sorted(freqDist_oftwoWords.most_common(100), key = lambda x: x[1], reverse = True)

# создаём набор слов для "двухсловок"
# Так как тип данных "словарное множество" то извлечем Топ-100 слов которые наиболее часто встречаются в запросах ключевых слов
twoBag_words = [keyword[0] for keyword in twoBag_words]

In [ ]:
# Создаем аггрегированные данные 
a = []
for b in twoBag_words:
  b1, b2 = b.split(' ')
  a.append((b, checking_data[checking_data['keywords'].str.contains(b1) | checking_data['keywords'].str.contains(b2)][['impressions', 'clicks']].sum().values[0], 
               checking_data[checking_data['keywords'].str.contains(b1) | checking_data['keywords'].str.contains(b2)][['impressions', 'clicks']].sum().values[1]))

# набор агрегированных данные
groupedTwo_df = pd.DataFrame.from_records(a, columns = ['keywords', 'impressions', 'clicks'])

In [ ]:
# группируем однословные частоты в набор и готово
groupedTwo_df = groupedTwo_df.sort_values(by = 'clicks', ascending = False)

In [ ]:
# создаём набор слов для "трехсловок"
# Так как тип данных "словарное множество" то извлечем Топ-100 слов которые наиболее часто встречаются в запросах ключевых слов
threeBag_words = sorted(freqDist_ofthreeWords.most_common(100), key = lambda x: x[1], reverse = True)

# Переводим в массив
threeBag_words = [keyword[0] for keyword in threeBag_words]

In [ ]:
# Создаем аггрегированные данные 
a = []
for b in threeBag_words:
  b1, b2, b3 = b.split(' ')
  a.append((b, checking_data[checking_data['keywords'].str.contains(b1) | checking_data['keywords'].str.contains(b2) | checking_data['keywords'].str.contains(b3)][['impressions', 'clicks']].sum().values[0], 
               checking_data[checking_data['keywords'].str.contains(b1) | checking_data['keywords'].str.contains(b2) | checking_data['keywords'].str.contains(b3)][['impressions', 'clicks']].sum().values[1],
))

# набор агрегированных данные
groupedThree_df = pd.DataFrame.from_records(a, columns = ['keywords', 'impressions', 'clicks'])

In [ ]:
# группируем однословные частоты в набор и готово
groupedThree_df = groupedThree_df.sort_values(by = 'clicks', ascending = False)

In [ ]:
# создаём набор слов для "четырехсловок"
# Так как тип данных "словарное множество" то извлечем Топ-100 слов которые наиболее часто встречаются в запросах ключевых слов
fourBag_words = sorted(freqDist_offourWords.most_common(100), key = lambda x: x[1], reverse = True)

# Переводим в массив
fourBag_words = [keyword[0] for keyword in fourBag_words]

In [ ]:
# Создаем аггрегированные данные 
a = []
for b in fourBag_words:
  b1, b2, b3, b4 = b.split(' ')
  a.append((b, checking_data[checking_data['keywords'].str.contains(b1) | checking_data['keywords'].str.contains(b2) | checking_data['keywords'].str.contains(b3) | checking_data['keywords'].str.contains(b4)][['impressions', 'clicks']].sum().values[0], 
               checking_data[checking_data['keywords'].str.contains(b1) | checking_data['keywords'].str.contains(b2) | checking_data['keywords'].str.contains(b3) | checking_data['keywords'].str.contains(b4)][['impressions', 'clicks']].sum().values[1]))

# набор агрегированных данные
groupedFour_df = pd.DataFrame.from_records(a, columns = ['keywords', 'impressions', 'clicks'])

In [ ]:
groupedFour_df = groupedFour_df.sort_values(by = 'clicks', ascending = False)

In [ ]:
# создаём набор слов для "пятисловок"
# Так как тип данных "словарное множество" то извлечем Топ-100 слов которые наиболее часто встречаются в запросах ключевых слов
fiveBag_words = sorted(freqDist_offiveWords.most_common(100), key = lambda x: x[1], reverse = True)

# Переводим в массив
fiveBag_words = [keyword[0] for keyword in fiveBag_words]

In [ ]:
# Создаем аггрегированные данные 
a = []
for b in fiveBag_words:
  b1, b2, b3, b4, b5 = b.split(' ')
  a.append((b, checking_data[checking_data['keywords'].str.contains(b1) | checking_data['keywords'].str.contains(b2) | checking_data['keywords'].str.contains(b3) | checking_data['keywords'].str.contains(b4) | checking_data['keywords'].str.contains(b5)][['impressions', 'clicks']].sum().values[0], 
               checking_data[checking_data['keywords'].str.contains(b1) | checking_data['keywords'].str.contains(b2) | checking_data['keywords'].str.contains(b3) | checking_data['keywords'].str.contains(b4) | checking_data['keywords'].str.contains(b5)][['impressions', 'clicks']].sum().values[1]))

# набор агрегированных данные
groupedFive_df = pd.DataFrame.from_records(a, columns = ['keywords', 'impressions', 'clicks'])

In [ ]:
groupedFive_df = groupedFive_df.sort_values(by = 'clicks', ascending = False)

In [ ]:
# создаём набор слов для "шестисловок"
# Так как тип данных "словарное множество" то извлечем Топ-100 слов которые наиболее часто встречаются в запросах ключевых слов
sixBag_words = sorted(freqDist_ofsixWords.most_common(100), key = lambda x: x[1], reverse = True)

# Переводим в массив
sixBag_words = [keyword[0] for keyword in sixBag_words]

In [ ]:
# Создаем аггрегированные данные 
a = []
for b in sixBag_words:
  b1, b2, b3, b4, b5, b6 = b.split(' ')
  a.append((b, checking_data[checking_data['keywords'].str.contains(b1) | checking_data['keywords'].str.contains(b2) | checking_data['keywords'].str.contains(b3) | checking_data['keywords'].str.contains(b4) | checking_data['keywords'].str.contains(b5) | checking_data['keywords'].str.contains(b6)][['impressions', 'clicks']].sum().values[0], 
               checking_data[checking_data['keywords'].str.contains(b1) | checking_data['keywords'].str.contains(b2) | checking_data['keywords'].str.contains(b3) | checking_data['keywords'].str.contains(b4) | checking_data['keywords'].str.contains(b5) | checking_data['keywords'].str.contains(b6)][['impressions', 'clicks']].sum().values[1]))

# набор агрегированных данные
groupedSix_df = pd.DataFrame.from_records(a, columns = ['keywords', 'impressions', 'clicks'])

In [ ]:
groupedSix_df = groupedSix_df.sort_values(by = 'clicks', ascending = False)

In [ ]:
export_DocGroup = [groupedOne_df, groupedTwo_df, groupedThree_df, groupedFour_df, groupedFive_df, groupedSix_df]

# Экспортируем полученные результаты по частотности в файлы csv для передачи отделам
def to_exportDocs(data_to_export):
  for num, document in enumerate(data_to_export):
    document.to_csv('drive/My Drive/Colab Notebooks/KeywordsAnalytics_KeywordsModelling/Exported_GroupedDocuments_NonUniqueSet/'+ str(num) + '_Groupeddocument.csv', index = False)

In [ ]:
# Экспортируем данные в папку
# Данные представляют с собой наиболее часто используемые однословники, двух... и ...шестисловники в наборе данных
# То есть, те keywords которые были использованны в процессе работы чаще всего.
to_exportDocs(export_DocGroup)

##### По прямому вхождению

In [ ]:
# Проводим аналогичный процесс только все данные теперь будут inclusive то есть уникальные сеты

# Создаем аггрегированные данные по массиву частотности однословок
a = []
for b in bag_words:
  a.append((b, checking_data[checking_data['keywords'].str.contains(b)][['impressions', 'clicks']].sum().values[0], 
               checking_data[checking_data['keywords'].str.contains(b)][['impressions', 'clicks']].sum().values[1]))

# набор агрегированных данные
groupedUniqueOne_df = pd.DataFrame.from_records(a, columns = ['keywords', 'impressions', 'clicks'])

# группируем однословные частоты в набор и готово
groupedUniqueOne_df = groupedUniqueOne_df.sort_values(by = 'clicks', ascending = False)[:100]

In [ ]:
# Создаем аггрегированные данные 
a = []
for b in twoBag_words:
  b1, b2 = b.split(' ')
  a.append((b, checking_data[checking_data['keywords'].str.contains(b1) & checking_data['keywords'].str.contains(b2)][['impressions', 'clicks']].sum().values[0], 
               checking_data[checking_data['keywords'].str.contains(b1) & checking_data['keywords'].str.contains(b2)][['impressions', 'clicks']].sum().values[1]))

# набор агрегированных данные
groupedUniqueTwo_df = pd.DataFrame.from_records(a, columns = ['keywords', 'impressions', 'clicks'])
# группируем однословные частоты в набор и готово
groupedUniqueTwo_df = groupedUniqueTwo_df.sort_values(by = 'clicks', ascending = False)

In [ ]:
# Создаем аггрегированные данные 
a = []
for b in threeBag_words:
  b1, b2, b3 = b.split(' ')
  a.append((b, checking_data[checking_data['keywords'].str.contains(b1) & checking_data['keywords'].str.contains(b2) & checking_data['keywords'].str.contains(b3)][['impressions', 'clicks']].sum().values[0], 
               checking_data[checking_data['keywords'].str.contains(b1) & checking_data['keywords'].str.contains(b2) & checking_data['keywords'].str.contains(b3)][['impressions', 'clicks']].sum().values[1]))

# набор агрегированных данные
groupedUniqueThree_df = pd.DataFrame.from_records(a, columns = ['keywords', 'impressions', 'clicks'])

# группируем однословные частоты в набор и готово
groupedUniqueThree_df = groupedUniqueThree_df.sort_values(by = 'clicks', ascending = False)

In [ ]:
# Создаем аггрегированные данные 
a = []
for b in fourBag_words:
  b1, b2, b3, b4 = b.split(' ')
  a.append((b, checking_data[checking_data['keywords'].str.contains(b1) & checking_data['keywords'].str.contains(b2) & checking_data['keywords'].str.contains(b3) & checking_data['keywords'].str.contains(b4)][['impressions', 'clicks']].sum().values[0], 
               checking_data[checking_data['keywords'].str.contains(b1) & checking_data['keywords'].str.contains(b2) & checking_data['keywords'].str.contains(b3) & checking_data['keywords'].str.contains(b4)][['impressions', 'clicks']].sum().values[1]))

# набор агрегированных данные
groupedUniqueFour_df = pd.DataFrame.from_records(a, columns = ['keywords', 'impressions', 'clicks'])

groupedUniqueFour_df = groupedUniqueFour_df.sort_values(by = 'clicks', ascending = False)

In [ ]:
# Создаем аггрегированные данные 
a = []
for b in fiveBag_words:
  b1, b2, b3, b4, b5 = b.split(' ')
  a.append((b, checking_data[checking_data['keywords'].str.contains(b1) & checking_data['keywords'].str.contains(b2) & checking_data['keywords'].str.contains(b3) & checking_data['keywords'].str.contains(b4) & checking_data['keywords'].str.contains(b5)][['impressions', 'clicks']].sum().values[0], 
               checking_data[checking_data['keywords'].str.contains(b1) & checking_data['keywords'].str.contains(b2) & checking_data['keywords'].str.contains(b3) & checking_data['keywords'].str.contains(b4) & checking_data['keywords'].str.contains(b5)][['impressions', 'clicks']].sum().values[1]))

# набор агрегированных данные
groupedUniqueFive_df = pd.DataFrame.from_records(a, columns = ['keywords', 'impressions', 'clicks'])

groupedUniqueFive_df = groupedUniqueFive_df.sort_values(by = 'clicks', ascending = False)

In [ ]:
# Создаем аггрегированные данные 
a = []
for b in sixBag_words:
  b1, b2, b3, b4, b5, b6 = b.split(' ')
  a.append((b, checking_data[checking_data['keywords'].str.contains(b1) & checking_data['keywords'].str.contains(b2) & checking_data['keywords'].str.contains(b3) & checking_data['keywords'].str.contains(b4) & checking_data['keywords'].str.contains(b5) & checking_data['keywords'].str.contains(b6)][['impressions', 'clicks']].sum().values[0], 
               checking_data[checking_data['keywords'].str.contains(b1) & checking_data['keywords'].str.contains(b2) & checking_data['keywords'].str.contains(b3) & checking_data['keywords'].str.contains(b4) & checking_data['keywords'].str.contains(b5) & checking_data['keywords'].str.contains(b6)][['impressions', 'clicks']].sum().values[1]))

# набор агрегированных данные
groupedUniqueSix_df = pd.DataFrame.from_records(a, columns = ['keywords', 'impressions', 'clicks'])

groupedUniqueSix_df = groupedUniqueSix_df.sort_values(by = 'clicks', ascending = False)

In [ ]:
export_DocGroup = [groupedUniqueOne_df, groupedUniqueTwo_df, groupedUniqueThree_df, groupedUniqueFour_df, groupedUniqueFive_df, groupedUniqueSix_df]

# Экспортируем полученные результаты по частотности в файлы csv для передачи отделам
def to_exportDocs(data_to_export):
  for num, document in enumerate(data_to_export):
    document.to_csv('drive/My Drive/Colab Notebooks/KeywordsAnalytics_KeywordsModelling/Exported_GroupedDocuments_UniqueSet/'+ str(num) + '_GroupedUniqueDocument.csv', index = False)

In [ ]:
to_exportDocs(export_DocGroup)

### **Проверка частотности слов конкурентов.**

In [ ]:
# Цель провести частотность по использованию keywords для конкурентов
# Использоваться будет набор данных competitorUS в котором содержится информация о часто используемых keywords конкурентами в данной сфере
competitorUS.head()

,keywords
0,interracial dating sites
1,dating sites
2,dating for seniors
3,lds singles
4,interracial dating site


In [ ]:
# Фильтр данных на тип данных строковый объект
competitorUS['keywords'] = competitorUS['keywords'].astype(str)

# Первым делом переведем все слова находящиеся в колонке в нижний регистр
competitorUS['keywords'] = competitorUS['keywords'].str.lower()

# Вычисляем максимальное и минимальное словосочетание (количество слов в keyword sequence)
print('Максимальное словосочетание в наборе keywords {} слова.'.format(competitorUS['keywords'].str.split().str.len().max()))
print('Минимальное словосочетание в наборе keywords {} слово.'.format(competitorUS['keywords'].str.split().str.len().min()))

Максимальное словосочетание в наборе keywords 23 слова.
Минимальное словосочетание в наборе keywords 1 слово.


In [ ]:
# Избавляемся от пунктуации
competitorUS['keywords'] = competitorUS['keywords'].str.replace('[^\w\s]', '')

# Теперь удаляем все смайлики и прочую non-ASCII текст в наборе данных
competitorUS['keywords'] = competitorUS['keywords'].apply(emoji)

In [ ]:
# Проверка на пустые значения:
print('Количество пустых значений в наборе данных: {}'.format(len(competitorUS[competitorUS['keywords'] == ''])))

Количество пустых значений в наборе данных: 5


In [ ]:
# Удаляем такие значения и получаем готовый набор данных с которым можно работать
competitorUS = competitorUS[competitorUS['keywords'] != '']

In [ ]:
# Удаляем эмотиконы
competitorUS['keywords'] = competitorUS['keywords'].apply(remove_emoticons)

# Удаляем веб адреса если они присутствуют
competitorUS['keywords'] = competitorUS['keywords'].apply(remove_urls)

# Удаляем пунктуацию
competitorUS['keywords'] = competitorUS['keywords'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
competitorUS['keywords'] = competitorUS['keywords'].apply(lambda x: x.translate(str.maketrans('', '', '1234567890')))

# фильтр строк на пустые символы
competitorUS['keywords'].str.replace(' ', '')
competitorUS.keywords = competitorUS.keywords.str.strip()
# избавляемся от пустых строк
competitorUS = competitorUS[competitorUS['keywords'] != '']

In [ ]:
# Распознавание языков
langs = []
for sentence in competitorUS['keywords']:
  lang = model.predict(sentence)[0]
  langs.append(str(lang)[11:13])

competitorUS['language'] = langs

# избавляемся от текста, который имеют языковую разметку != en
len(competitorUS[competitorUS['language'] != 'en'])

9813

In [ ]:
competitorUS.language.value_counts(normalize = True)[:10]

en    0.932270
fr    0.013956
es    0.010353
de    0.008427
it    0.006943
ce    0.003058
pt    0.002913
nl    0.002216
ca    0.002188
sv    0.001698
Name: language, dtype: float64

In [ ]:
# набор данных, который содержит в себе только английские слова
competitorUS = competitorUS[competitorUS['language'] == 'en']
del(competitorUS['language'])

In [ ]:
# Токенизируем текстовые данные
# В предложения
competitors_tokenized_sentences = competitorUS['keywords'].tolist()

# В отдельные слова
# Тоже самое только надо воспроизвести отдельны "единичные слова" в наборе keywords в наборе данных
competitors_tokenized_words = competitorUS['keywords'].str.split(' ').explode().str.strip().tolist()

In [ ]:
import nltk
nltk.download('punkt')

# Убираем стоп слова с предложений по ключевым словам
competitors_filtered_sentences = []
for sentence in competitors_tokenized_sentences:
    competitors_filtered_sentences.append(' '.join(word for word in nltk.word_tokenize(sentence) if word.lower() not in stop_words))

competitors_filtered_sentences = [word for word in competitors_filtered_sentences if word]


# Убираем стоп слова с предложений по ключевым однословкам
competitors_filtered_words = []
for sentence in competitors_tokenized_words:
    competitors_filtered_words.append(''.join(word for word in nltk.word_tokenize(sentence) if word.lower() not in stop_words))

competitors_filtered_words = [word for word in competitors_filtered_words if word]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Создаем массивы для ключевых слов. Для 2, 3, 4 и 5 словосочетаний.
competitors_tokenized_twoWords = []
competitors_tokenized_threeWords = []
competitors_tokenized_fourWords = []
competitors_tokenized_fiveWords = []
competitors_tokenized_sixWords = []
competitors_tokenized_sevenWords = []
competitors_tokenized_eightWords = []
competitors_tokenized_nineWords = []
competitors_tokenized_tenWords = []

# Вызываем функцию для поиска и заполняем массив
structure_keyword(competitors_filtered_sentences, 2, competitors_tokenized_twoWords)
structure_keyword(competitors_filtered_sentences, 3, competitors_tokenized_threeWords)
structure_keyword(competitors_filtered_sentences, 4, competitors_tokenized_fourWords)
structure_keyword(competitors_filtered_sentences, 5, competitors_tokenized_fiveWords)
structure_keyword(competitors_filtered_sentences, 6, competitors_tokenized_sixWords)
structure_keyword(competitors_filtered_sentences, 7, competitors_tokenized_sevenWords)
structure_keyword(competitors_filtered_sentences, 8, competitors_tokenized_eightWords)
structure_keyword(competitors_filtered_sentences, 9, competitors_tokenized_nineWords)
structure_keyword(competitors_filtered_sentences, 10, competitors_tokenized_tenWords)

print(' Объем каждого из массивов:\n 2 keywords: {}\n 3 keywords: {}\n 4 keywords: {}\n 5 keywords: {}\n 6 keywords: {}\n 7 keywords: {}\n 8 keywords: {}\n 9 keywords: {}\n 10 keywords: {}\n'.format(
                                                                                                                 len(competitors_filtered_words),
                                                                                                                 len(competitors_tokenized_twoWords),
                                                                                                                 len(competitors_tokenized_threeWords),
                                                                                                                 len(competitors_tokenized_fourWords),
                                                                                                                 len(competitors_tokenized_fiveWords),
                                                                                                                 len(competitors_tokenized_sixWords),
                                                                                                                 len(competitors_tokenized_sevenWords),
                                                                                                                 len(competitors_tokenized_eightWords),
                                                                                                                 len(competitors_tokenized_nineWords),
                                                                                                                 len(competitors_tokenized_tenWords)))

 Объем каждого из массивов:
 2 keywords: 410317
 3 keywords: 33467
 4 keywords: 57875
 5 keywords: 30208
 6 keywords: 7062
 7 keywords: 1119
 8 keywords: 197
 9 keywords: 45
 10 keywords: 19



In [ ]:
# One word Distribution
competitors_freq_OnewordDist = FreqDist(competitors_filtered_words)

# Sentence Distribution
competitors_freq_SentDist = FreqDist(competitors_tokenized_sentences)

# Most common Two Words Distribution
competitors_freqDist_oftwoWords = FreqDist(competitors_tokenized_twoWords)

# Most common Three Words Distribution
competitors_freqDist_ofthreeWords = FreqDist(competitors_tokenized_threeWords)

# Most common Four Words Distribution
competitors_freqDist_offourWords = FreqDist(competitors_tokenized_fourWords)

# Most common Five Words Distribution
competitors_freqDist_offiveWords = FreqDist(competitors_tokenized_fiveWords)

# Most common Six Words Distribution
competitors_freqDist_ofsixWords =  FreqDist(competitors_tokenized_sixWords)

In [ ]:
export_data = [competitors_freq_OnewordDist, competitors_freqDist_oftwoWords, competitors_freqDist_ofthreeWords, competitors_freqDist_offourWords, competitors_freqDist_offiveWords, competitors_freqDist_ofsixWords]

# Экспортируем полученные результаты по частотности в файлы csv для передачи отделам
def to_export(data_to_export):
  for num, document in enumerate(data_to_export):
    export_doc = pd.DataFrame(document.most_common(100), columns = ['keywords', 'frequency']).sort_values(by = ['frequency'], ascending = False)
    export_doc.to_csv('drive/My Drive/Colab Notebooks/KeywordsAnalytics_KeywordsModelling/Exported_keywordFrequency_Competitors/'+ str(num) + '_documentFreqency.csv', index = False)

In [ ]:
# Export Data To Google Drive Cloud
to_export(export_data)

### **Выбор уникальных ключевых слов у компетиторов, которыми не пользуемся мы. И наоборот.**

In [ ]:
# Формирование однословок и пересечение с нашими однословками

# Массив однословок конкурентов
# Наш массив однословок -> bag_words
competitors_OneWord_bagWords = sorted(competitors_freq_OnewordDist.items(), key = lambda x: x[1], reverse = True)
competitors_OneWord_bagWords  = [keyword[0] for keyword in competitors_OneWord_bagWords[:100]]

# Массив двухсловок конкурентов
# Наш массив двухсловок -> twoBag_words
competitors_TwoWord_bagWords = sorted(competitors_freqDist_oftwoWords.items(), key = lambda x: x[1], reverse = True)
competitors_TwoWord_bagWords  = [keyword[0] for keyword in competitors_TwoWord_bagWords[:100]]

# Массив Трехсловок конкурентов
# Наш массив трехсловок -> threeBag_words
competitors_ThreeWord_bagWords = sorted(competitors_freqDist_ofthreeWords.items(), key = lambda x: x[1], reverse = True)
competitors_ThreeWord_bagWords  = [keyword[0] for keyword in competitors_ThreeWord_bagWords[:100]]

# Массив Четырехловок конкурентов
# Наш массив четырехсловок -> fourBag_words
competitors_FourWord_bagWords = sorted(competitors_freqDist_offourWords.items(), key = lambda x: x[1], reverse = True)
competitors_FourWord_bagWords = [keyword[0] for keyword in competitors_FourWord_bagWords[:100]]

# Массив Пятисловок конкурентов
# Наш массив пятисловок -> fiveBag_words
competitors_FiveWord_bagwords = sorted(competitors_freqDist_offiveWords.items(), key = lambda x: x[1], reverse = True)
competitors_FiveWord_bagwords = [keyword[0] for keyword in competitors_FiveWord_bagwords[:100]]

# Массив Шестисловок конкурентов
# Наш массив шестисловок -> sixBag_words
competitors_SixWord_bagwords = sorted(competitors_freqDist_ofsixWords.items(), key = lambda x: x[1], reverse = True)
competitors_SixWord_bagwords = [keyword[0] for keyword in competitors_SixWord_bagwords[:100]]

In [ ]:
# Ищем пересечения в различных комбинациях
def intersection_words(listOne, listTwo):
  return list(set(listOne) & set(listTwo))

In [ ]:
# Общие однословки у нас и у конкурентов
one_wordCommon = intersection_words(bag_words, competitors_OneWord_bagWords)

# Общие двухсловки у нас и у конкурентов
two_wordCommon = intersection_words(twoBag_words, competitors_TwoWord_bagWords)

# Общие трехсловки у нас и у конкурентов
three_wordCommon = intersection_words(threeBag_words, competitors_ThreeWord_bagWords)

# Общие четырехсловки у нас и у конкурентов
four_wordCommon = intersection_words(fourBag_words, competitors_FourWord_bagWords)

# Общие пятисловки у нас и у конкурентов
five_wordCommon = intersection_words(fiveBag_words, competitors_FiveWord_bagwords)

# Общие шестисловки у нас и у конкурентов
six_wordCommon = intersection_words(sixBag_words, competitors_SixWord_bagwords)

In [ ]:
export_data = [one_wordCommon, two_wordCommon, three_wordCommon, four_wordCommon, five_wordCommon, six_wordCommon]

# Экспортируем полученные результаты по частотности в файлы csv для передачи отделам
def to_export(data_to_export):
  for num, document in enumerate(data_to_export):
    export_doc = pd.DataFrame(document, columns = ['common_keywords'])
    export_doc.to_csv('drive/My Drive/Colab Notebooks/KeywordsAnalytics_KeywordsModelling/Exported_KeywordsIntersection_withCompetitors/'+ str(num) + '_documentCommonKeywords.csv', index = False)

to_export(export_data)

In [ ]:
# Поиск уникальных однословок
# Положение аргументов имеет значение/ Первый аргумент функции - основной массив на котором вычисляются уникальные ключевые слова
def keywordDifference(listOne, listTwo):
  return list(set(listOne) - (set(listTwo)))

In [ ]:
# Наши Уникальные однословки, которых нет у конкурентов
one_wordUnique_ourKeywords = keywordDifference(bag_words, competitors_OneWord_bagWords)

# Наши Уникальные двухсловки, которых нет у конкурентов
two_wordUnique_ourKeywords = keywordDifference(twoBag_words, competitors_TwoWord_bagWords)

# Наши Уникальные трехсловки, которых нет у конкурентов
three_wordUnique_ourKeywords = keywordDifference(threeBag_words, competitors_ThreeWord_bagWords)

# Наши Уникальные четырехсловки, которых нет у конкурентов
four_wordUnique_ourKeywords = keywordDifference(fourBag_words, competitors_FourWord_bagWords)

# Наши Уникальные пятисловки, которых нет у конкурентов
five_wordUnique_ourKeywords = keywordDifference(fiveBag_words, competitors_FiveWord_bagwords)

# Наши Уникальные шестисловки, которых нет у конкурентов
six_wordUnique_ourKeywords = keywordDifference(sixBag_words, competitors_SixWord_bagwords)

In [ ]:
export_data = [one_wordUnique_ourKeywords, two_wordUnique_ourKeywords, three_wordUnique_ourKeywords, four_wordUnique_ourKeywords, five_wordUnique_ourKeywords, six_wordUnique_ourKeywords]

# Экспортируем полученные результаты по частотности в файлы csv для передачи отделам
def to_export(data_to_export):
  for num, document in enumerate(data_to_export):
    export_doc = pd.DataFrame(document, columns = ['ourUnique_keywords'])
    export_doc.to_csv('drive/My Drive/Colab Notebooks/KeywordsAnalytics_KeywordsModelling/Exported_UniqueKeyword_Ours/'+ str(num) + '_documentUniqueKeywords.csv', index = False)

to_export(export_data)

In [ ]:
# Для поиска уникальных слов конкурентов используем ту же функцию только меняем аргументы функции местами
def keywordDifference(listOne, listTwo):
  return list(set(listOne) - (set(listTwo)))

# Уникальные однословки конкурента
one_wordUnique_CompetitorsKeywords = keywordDifference(competitors_OneWord_bagWords, bag_words)

# Уникальные двусловки конкурента
two_wordUnique_CompetitorsKeywords = keywordDifference(competitors_TwoWord_bagWords, twoBag_words)

# Уникальные трехсловки конкурента
three_wordUnique_CompetitorsKeywords = keywordDifference(competitors_ThreeWord_bagWords, threeBag_words)

# Уникальные четырехсловки конкурента
four_wordUnique_CompetitorsKeywords = keywordDifference(competitors_FourWord_bagWords, fourBag_words)

# Уникальные пятисловки конкурента
five_wordUnique_CompetitorsKeywords = keywordDifference(competitors_FiveWord_bagwords, fiveBag_words)

# Уникальные шестисловки конкурента
six_wordUnique_CompetitorsKeywords = keywordDifference(competitors_SixWord_bagwords, sixBag_words)

In [ ]:
export_data = [one_wordUnique_CompetitorsKeywords, two_wordUnique_CompetitorsKeywords, three_wordUnique_CompetitorsKeywords, four_wordUnique_CompetitorsKeywords, five_wordUnique_CompetitorsKeywords, six_wordUnique_CompetitorsKeywords]

# Экспортируем полученные результаты по частотности в файлы csv для передачи отделам
def to_export(data_to_export):
  for num, document in enumerate(data_to_export):
    export_doc = pd.DataFrame(document, columns = ['CompetitorsUnique_keywords'])
    export_doc.to_csv('drive/My Drive/Colab Notebooks/KeywordsAnalytics_KeywordsModelling/Exported_UniqueKeyword_Competitors/'+ str(num) + '_documentUniqueKeywords.csv', index = False)

to_export(export_data)

### **KeyWord Generation из массива однословок (наши уникальные и уникальные конкурентов). Проверка Search Volume сгенерированных ключевых слов.**

In [ ]:
# Из уникальных и часто ипользуемых однословок генерируем комбинации всех слов длинной до 2, 3, 4 и 5 слов в комбинации
import itertools

# Наши сгенерированные ключевые слова с длинной строки 2 слова
our_uniqueTwoWord_keywords =  list(itertools.combinations(one_wordUnique_ourKeywords, 2))
# Сгенерированные ключевые слова конкурентов с длинной строки 2 слова
competitors_uniqueTwoWord_keywords = list(itertools.combinations(one_wordUnique_CompetitorsKeywords, 2))


# Наши сгенерированные ключевые слова с длинной строки 3 слова
our_uniqueThreeWord_keywords =  list(itertools.combinations(one_wordUnique_ourKeywords, 3))
# Сгенерированные ключевые слова конкурентов с длинной строки 3 слова
competitors_uniqueThreeWord_keywords = list(itertools.combinations(one_wordUnique_CompetitorsKeywords, 3))

# Наши сгенерированные ключевые слова с длинной строки 4 слова
our_uniqueFourWord_keywords =  list(itertools.combinations(one_wordUnique_ourKeywords, 4))
# Сгенерированные ключевые слова конкурентов с длинной строки 4 слова
competitors_uniqueFourWord_keywords = list(itertools.combinations(one_wordUnique_CompetitorsKeywords, 4))

# Наши сгенерированные ключевые слова с длинной строки 5 слов
our_uniqueFiveWord_keywords =  list(itertools.combinations(one_wordUnique_ourKeywords, 5))
# Сгенерированные ключевые слова конкурентов с длинной строки 5 слова
competitors_uniqueFiveWord_keywords = list(itertools.combinations(one_wordUnique_CompetitorsKeywords, 5))

#=============================================================================================================

# Создаем массив со сгенерированными ключевыми словами (длина 2 слова)
our_uniqueTwoWord_keywords = [' '.join(ngram) for ngram in our_uniqueTwoWord_keywords]
# Аналогично для конкурентов
competitors_uniqueTwoWord_keywords = [' '.join(ngram) for ngram in competitors_uniqueTwoWord_keywords]

# Создаем массив со сгенерированными ключевыми словами (длина 3 слова)
our_uniqueThreeWord_keywords = [' '.join(ngram) for ngram in our_uniqueThreeWord_keywords]
# Аналогично для конкурентов
competitors_uniqueThreeWord_keywords = [' '.join(ngram) for ngram in competitors_uniqueThreeWord_keywords]

# Создаем массив со сгенерированными ключевыми словами (длина 4 слова)
our_uniqueFourWord_keywords = [' '.join(ngram) for ngram in our_uniqueFourWord_keywords]
# Аналогично для конкурентов
competitors_uniqueFourWord_keywords = [' '.join(ngram) for ngram in competitors_uniqueFourWord_keywords]

# Создаем массив со сгенерированными ключевыми словами (длинна 5 слов)
our_uniqueFiveWord_keywords = [' '.join(ngram) for ngram in our_uniqueFiveWord_keywords]
# Аналогично для конкурентов
competitors_uniqueFiveWord_keywords = [' '.join(ngram) for ngram in competitors_uniqueFiveWord_keywords]

#==============================================================================================================

# Объединяем два массива в один, чтобы получить полный массив сгенерированных ключевых слов 
all_keywords_array = our_uniqueTwoWord_keywords + competitors_uniqueTwoWord_keywords + our_uniqueThreeWord_keywords + competitors_uniqueThreeWord_keywords + our_uniqueFourWord_keywords + competitors_uniqueFourWord_keywords + our_uniqueFiveWord_keywords + competitors_uniqueFiveWord_keywords

twoWords_array = our_uniqueTwoWord_keywords + competitors_uniqueTwoWord_keywords
threeWords_array = our_uniqueThreeWord_keywords + competitors_uniqueThreeWord_keywords
fourWords_array = our_uniqueFourWord_keywords + competitors_uniqueFourWord_keywords
fiveWords_array = our_uniqueFiveWord_keywords + competitors_uniqueFiveWord_keywords

In [ ]:
print('Длина массива сгенерированных слов: {} комбинации.'.format(len(all_keywords_array)))

Длина массива сгенерированных слов: 4739770 комбинации.


In [ ]:
# Выводим всё в набор данных
all_keywords = pd.DataFrame(all_keywords_array, columns = ['keywords_generated'])

twoWords = pd.DataFrame(twoWords_array, columns = ['keywords_generated'])
threeWords_array = pd.DataFrame(threeWords_array, columns = ['keywords_generated'])
fourWords_array = pd.DataFrame(fourWords_array, columns = ['keywords_generated'])
fiveWords_array = pd.DataFrame(fiveWords_array, columns = ['keywords_generated'])

In [ ]:
# Сохраняем на всякий случай
all_keywords.to_csv('drive/My Drive/Colab Notebooks/KeywordsAnalytics_KeywordsModelling/ExportedGenerated_KeyWords/GeneratedKeywords.csv', index = False)

In [ ]:
twoWords.to_csv('drive/My Drive/Colab Notebooks/KeywordsAnalytics_KeywordsModelling/ExportedGenerated_KeyWords/TwoWordsGeneratedKeywords.csv', index = False)
threeWords_array.to_csv('drive/My Drive/Colab Notebooks/KeywordsAnalytics_KeywordsModelling/ExportedGenerated_KeyWords/ThreeWordsGeneratedKeywords.csv', index = False)
fourWords_array.to_csv('drive/My Drive/Colab Notebooks/KeywordsAnalytics_KeywordsModelling/ExportedGenerated_KeyWords/FourWordsGeneratedKeywords.csv', index = False)
fiveWords_array.to_csv('drive/My Drive/Colab Notebooks/KeywordsAnalytics_KeywordsModelling/ExportedGenerated_KeyWords/FiveWordsGeneratedKeywords.csv', index = False)

In [ ]:
print('2:{}   3:{}   4:{}  5:{}'.format(len(twoWords), len(threeWords_array),len(fourWords_array), len(fiveWords_array)))

2:2450   3:39200   4:460600  5:4237520


### **Настройка окружения для селектора и подключения к API сервисам Google. Запускать но не изменять.**

In [ ]:
# For OAuth 2.0 
# Client ID: 568960871098-235t01i3v9ol4rrafjbj1l0d7vl15rho.apps.googleusercontent.com
# Client Secret: vOZ7SQl2kbj2xW9EOb6n-j2G


# For Google Adwords API
# email: mguzevich@avanta.co
# API Token: _TyS_IZDspl4Sd_Q4E1bsQ


# Refreshed Token after autherisation
# 1//09ObEJnvjDbLICgYIARAAGAkSNwF-L9IrjGySBnzDtWNhMpwsYFiDB85IFhRxOk34_XYEjAT0B2dPwPkWpOTPNhRQ7MDI5ipxcbE

# Access Token
# ya29.a0AfH6SMAv4EWwyfrnBNRX550YXFYsQorrC8zjJumjbw-gwSTCz3SFldl78vMTuaUXfqOv1-oivTowcRA0ybJpSeOWvi6o_r2ZYKsmc-6in0Lj2fRixLdk1ExpYiJU-__qkyGo9cwpxabOWIWx607zNzAMxLZVpPDKtMo




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%ls -la /content/drive/My\ Drive/Colab\ Notebooks/KeywordsAnalytics_KeywordsModelling

total 1029
drwx------ 2 root root    4096 Jul  7 08:37 Data/
drwx------ 2 root root    4096 Jul 17 09:39 Exorted_GoogleTrendsCheck/
drwx------ 2 root root    4096 Jul 16 13:20 ExportedGenerated_KeyWords/
drwx------ 2 root root    4096 Jul 14 10:33 Exported_GroupedDocuments_NonUniqueSet/
drwx------ 2 root root    4096 Jul 14 11:03 Exported_GroupedDocuments_UniqueSet/
drwx------ 2 root root    4096 Jul 14 11:53 Exported_keywordFrequency_Competitors/
drwx------ 2 root root    4096 Jul 14 10:33 Exported_keywordsFrequency_UniqueSet/
drwx------ 2 root root    4096 Jul 15 09:50 Exported_KeywordsIntersection_withCompetitors/
drwx------ 2 root root    4096 Jul 15 11:34 Exported_UniqueKeyword_Competitors/
drwx------ 2 root root    4096 Jul 15 11:24 Exported_UniqueKeyword_Ours/
-rw------- 1 root root    6052 Jul 20 12:09 generate_refresh_token.py
-rw------- 1 root root 1006437 Jul 20 12:36 KeywordsAnalytics_KeywordsModelling.ipynb


In [ ]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/KeywordsAnalytics_KeywordsModelling

/content/drive/My Drive/Colab Notebooks/KeywordsAnalytics_KeywordsModelling


In [ ]:
%pwd

'/content/drive/My Drive/Colab Notebooks/KeywordsAnalytics_KeywordsModelling'

In [ ]:
# os.chdir('/content/drive/My Drive/Colab Notebooks/KeywordsAnalytics_KeywordsModelling')
# !pwd
!pip install googleads pandas
!pip install google-auth google-auth-oauthlib
import _locale
import googleads
import pandas as pd
import traceback
from googleads import adwords
from googleads import oauth2
_locale._getdefaultlocale = (lambda *args: ['en_UK', 'UTF-8'])

In [ ]:
!python generate_refresh_token.py

In [ ]:
!python generate_refresh_token.py '568960871098-96jkkgg5dfr4pcnaguntvse6b25aqbck.apps.googleusercontent.com' INSERT_CLIENT_ID 'h300g8wxGYkl086mwZZ-_js3' INSERT_CLIENT_SECRET 

In [ ]:
import _locale
import googleads
import traceback
from googleads import adwords
from googleads import oauth2

_locale._getdefaultlocale = (lambda *args: ['en_UK', 'UTF-8'])

# Class for searchVolume Settings 
class searchVolumePuller():
  def __init__(self, client_ID, client_secret, refresh_token, developer_token, client_customer_id):
    self.client_ID = client_ID
    self.client_secret = client_secret 
    self.refresh_token = refresh_token
    self.developer_token = developer_token
    self.client_customer_id = client_customer_id

    def get_client(self):
      access_token = oauth2.GoogleRefreshTokenClient(self.client_ID, self.client_secret, self.refresh_token)
      adwords_client = adwords.AdwordsClient(self.developer_token, access_token, client_customer_id = self.client_customer_id,
                                             cache = googleads.common.ZeepServiceProxy.NO_CACHE)
      return adwords_client

    def get_service(self, service, client):
      return client.GetService(service)

    
    # Configure selector for API request
    def get_search_volume(self, service_client, keyword_list):
      # empty dataframe to append data into and keywords and search volume lists
      keywords = []
      search_volume = []
      keywords_and_search_volume = pd.DataFrame()
      # split data into lists of 700 rows in order not to get banned from Google Adwords and Google API Services
      sublists = [keyword_list[x:x + 700] for x in range(0, len(keyword_list), 700)]
      for sublist in sublists:
        # Construct selector and get keyword stats.
        selector = {
            'ideaType' : 'KEYWORD',
            'requestType' : 'STATS'
        }
        # select attributes we want to retrieve
        selector['requestedAttributeTypes'] = ['KEYWORD_TEXT', 'SEARCH_VOLUME']

        # configuration of selectors paging limit to limit number of results
        offset = 0
        selector['paging'] = {
            'startIndex' : str(offset),
            'numberResults' : str(len(sublist))
        }

        # specify selectors keywords to suggest for
        selector['searchParameters'] = [{
            'xsi_type' : 'RelatedToQuerySearchParameter',
            'queries' : sublist
        }]

        # pull the data
        page = service_client.get(selector)
        # access json elements to return the suggestions
        for i in range(0, len(page['entries'])):
          keywords.append(page['entries'][i]['data'][0]['value']['value'])
          search_volume.append(page['entries'][i]['data'][1]['value']['value'])

        keywords_and_search_volume['Keywords'] = keywords
        keywords_and_search_volume['Search Volume'] = search_volume

        return keywords_and_search_volume
